In [2]:
from pyspark.sql import SparkSession, DataFrame, Window
import pyspark.sql.functions as F
import warnings
warnings.filterwarnings('ignore')

In [3]:
def read_college_date(spark: SparkSession, data_location: str = '../data/*.csv') -> DataFrame:
    df = spark.read.csv(data_location, header='true')
    return df

In [4]:
spark = SparkSession.builder.appName('HistoricCollegeData') \
        .getOrCreate()

21/08/10 22:56:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/08/10 22:56:17 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
df = read_college_date(spark=spark)

In [ ]:
df.limit(5).toPandas()

In [6]:
results = df.groupBy('INSTNM').agg(F.count('INSTNM').alias('UniqueInstituitionCount'))
results.sort(F.col('UniqueInstituitionCount').desc()).show(20)


In [23]:
# Which States has the highest number of student?
def pull_number_of_student_per_states(df: DataFrame):
    win = Window.partitionBy('STABBR').orderBy(F.col('NUM4_PUB').desc())
    df = df.withColumn('rowNum', F.row_number().over(win))
    df = df.groupby('STABBR').agg(F.sum(F.col('NUM4_PUB')).alias('total_number_of_students'))
    df.sort(F.col('total_number_of_students').desc()).limit(10).show()

# pull_number_of_student_per_states(df)

+------+------------------------+
|STABBR|total_number_of_students|
+------+------------------------+
|    CA|               1154205.0|
|    TX|                881766.0|
|    NY|                814605.0|
|    FL|                557794.0|
|    OH|                471870.0|
|    PA|                432544.0|
|    GA|                397956.0|
|    MI|                376380.0|
|    NC|                344533.0|
|    IL|                341352.0|
+------+------------------------+



In [31]:
def pull_avg_number_of_students_public_instituition(df: DataFrame):
    win = Window.partitionBy('INSTNM').orderBy(F.col('NUM4_PUB').desc())
    df = df.withColumn('rowNum', F.row_number().over(win))
    results = df.groupBy('INSTNM').agg(F.avg('NUM4_PUB').alias('avg_number_student_count'))
    results.sort(F.col('avg_number_student_count').desc()).show(20)

def pull_avg_number_of_students_private_instituition(df: DataFrame):
    win = Window.partitionBy('INSTNM').orderBy(F.col('NUM4_PRIV').desc())
    df = df.withColumn('rowNum', F.row_number().over(win))
    results = df.groupBy('INSTNM').agg(F.avg('NUM4_PRIV').alias('avg_number_student_count_private'))
    results.sort(F.col('avg_number_student_count_private').desc()).show(20)


In [38]:
pull_avg_number_of_students_private_instituition(df)

--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib64/python3.9/logging/__init__.py", line 1083, in emit
    msg = self.format(record)
  File "/usr/lib64/python3.9/logging/__init__.py", line 927, in format
    return fmt.format(record)
  File "/usr/lib64/python3.9/logging/__init__.py", line 666, in format
    s = self.formatMessage(record)
  File "/usr/lib64/python3.9/logging/__init__.py", line 635, in formatMessage
    return self._style.format(record)
  File "/usr/lib64/python3.9/logging/__init__.py", line 434, in format
    return self._format(record)
  File "/usr/lib64/python3.9/logging/__init__.py", line 430, in _format
    return self._fmt % record.__dict__
ValueError: unsupported format character '%' (0x25) at index 13
Call stack:
  File "/usr/lib64/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib64/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/home/Abuton

+--------------------+--------------------------------+
|              INSTNM|avg_number_student_count_private|
+--------------------+--------------------------------+
|University of Pho...|                         12314.0|
|University of Pho...|               5538.666666666667|
|Ultimate Medical ...|                          5038.0|
|Keiser University...|              2916.2727272727275|
|Full Sail University|              2684.3333333333335|
| New York University|               2645.818181818182|
|Colorado Technica...|              2356.8571428571427|
|  Liberty University|              2350.4545454545455|
|St. John's Univer...|              2308.3636363636365|
|Ultimate Medical ...|              2202.4444444444443|
|Rochester Institu...|                          2141.0|
|Universidad Ana G...|              2134.7272727272725|
| Syracuse University|              1943.8181818181818|
|   Baylor University|              1910.7272727272727|
|Everest Universit...|                         1

--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib64/python3.9/logging/__init__.py", line 1083, in emit
    msg = self.format(record)
  File "/usr/lib64/python3.9/logging/__init__.py", line 927, in format
    return fmt.format(record)
  File "/usr/lib64/python3.9/logging/__init__.py", line 666, in format
    s = self.formatMessage(record)
  File "/usr/lib64/python3.9/logging/__init__.py", line 635, in formatMessage
    return self._style.format(record)
  File "/usr/lib64/python3.9/logging/__init__.py", line 434, in format
    return self._format(record)
  File "/usr/lib64/python3.9/logging/__init__.py", line 430, in _format
    return self._fmt % record.__dict__
ValueError: unsupported format character '%' (0x25) at index 13
Call stack:
  File "/usr/lib64/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib64/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/home/Abuton